[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/Homeworks/blob/main/Part_1/Task_1.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"

In [3]:
# Импорт библиотек
import requests
import pandas as pd
from tqdm.auto import tqdm
import sqlite3
#from sqlalchemy import create_engine

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Ограничение вывода строк на экран
pd.options.display.max_rows = 20

## Список объектов

In [2]:
# Для скачивания данных по блоками по 100
# Задание параметров: начальное смещение равно 1, количество записей в блоке равно 100
offset_ = 0
limit_ = 100
# Пустой список ID объектов
objids = []
# Цикл из 1 млн итераций по 100 объектов (до 100млн домой, на сайте их 10583)
for i in range(1000000):
# Выполняем запрос к сайту, выдергиваем json
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        objects_data = res.json()
# Проверяем, есть ли в ответе объект
        objects_data.get('data').get('list')[0]
# Объекта нет - прерываем цикл
    except IndexError:
#        print('прерывание')
        break
# Не возникло исключение - берем ID объектов и добавляем их в список objids
    else:
        objects_list = objects_data.get('data').get('list')
        objids.extend([x.get('objId') for x in objects_list])
# увеличиваем смещение на 100
        offset_ += 100

In [3]:
# Определяем количество строящихся объектов на сайте
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset=0&limit=10&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
total = res.json().get('data').get('total')

# Задание параметров: начальное смещение равно 1, количество записей в блоке равно 100
offset_ = 0
limit_ = 100
# Пустой список ID объектов
objids = []
# Цикл выполняется, пока смещение не превысит общее количество объектов
while offset_ <= total:
# Выполняем запрос к сайту, выдергиваем json
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
# Берем ID объектов и добавляем их в список objids
    objects_list = res.json().get('data').get('list')
    objids.extend([x.get('objId') for x in objects_list])
# увеличиваем смещение на 100
    offset_ += 100

# Данные по определенному объекту

In [4]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
res_full = []
# Для каждого объекта в списке objids
for ID_obj in tqdm(objids):
# Получение информации
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{ID_obj}'
    res = requests.get(url)
# Добавление в итоговый список
    res_full.append(res.json().get('data'))
# СОздание датафрейма
df = pd.json_normalize(res_full)
df

100%|██████████| 33/33 [00:04<00:00,  7.71it/s]


,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,quartography.objLivElemCnt,quartography.objLivElemEntrCnt,quartography.objLivElemEntrFloorAvg,complexShortNm,greenAreaIndexValue,airQualityIndexValue,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,43207,295812,63,"г Самара, ул Маяковского","ЖК ""Маяковский NEW""",19,19,142,2024-09-30,Монолит-кирпич,...,142.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34040,232480,63,"г Самара, ул Советской Армии, д. 127","ЖК ""Журавли""",26,26,496,2023-06-30,Монолит-кирпич,...,496.0,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12643,25993,78,"г Санкт-Петербург, проспект Московский, д. 65,...",ЖК Promenade,17,17,0,2022-09-30,Другое,...,NaN,NaN,NaN,Promenade,2.0,NaN,NaN,NaN,NaN,NaN
3,31422,141383,42,"г Кемерово, Рудничный, микрорайон 14, д. 1/3","ЖК ""Юность-2""",18,18,144,2023-12-31,Панель,...,144.0,1.0,9.0,Юность,NaN,NaN,NaN,NaN,NaN,NaN
4,31530,151429,42,"г Кемерово, Рудничный, микрорайон 14, д. 3","ЖК ""Юность-2""",14,14,130,2022-12-31,Панель,...,130.0,2.0,5.0,Юность,NaN,NaN,NaN,NaN,NaN,NaN
5,31415,141360,42,"г Кемерово, Рудничный, микрорайон 14, д. 1/2","ЖК ""Юность-2""",18,18,144,2022-09-30,Панель,...,144.0,1.0,9.0,Юность,NaN,NaN,NaN,NaN,NaN,NaN
6,21592,54235,74,"г Челябинск, район Курчатовский, д. №3","Микрорайон «Парковый», 50 мкр. «Парковый»",17,17,225,2023-03-31,Панель,...,225.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,45277,490240,74,"п Западный, д. 1 (стр.)","""Белый Хутор""",6,7,97,2022-12-31,Панель,...,97.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5141,10461,74,"г Челябинск, Район Курчатовский, ул Александра...","Микрорайон № 50, Микрорайон «Парковый», 50 мкр...",17,17,90,2022-12-31,Блоки,...,90.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40251,461930,14,"г Якутск, улица Дежнева, д. 30",NaN,1,9,106,2024-03-31,Монолит-кирпич,...,106.0,2.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Сохранение результатов

In [5]:
# Сохранение результатов в excel
df.to_excel("ndrf_df.xlsx", encoding='cp1251')

In [6]:
# Сохранение результатов в pickle
df.to_pickle("ndrf_df.pkl")

In [180]:
# Создание базы данных SQLite
con = sqlite3.connect("ndrf_df.db")

# Приведение датафрейма к строковому типу (иначе SQL ругается)
df = df.astype('str')

# Сохранение датафрейма в таблицу buildings
df.to_sql("buildings", con, if_exists="replace", index=False)
con.commit()
con.close()


In [178]:
# Проверка того, как данные выгрузились в БД
df1 = pd.read_sql("SELECT * FROM buildings", con)
df1